In [1]:
import pandas as pd
import sqlalchemy as sql

/Users/kevinlee/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# connect to db
db_url='db/RT.sqlite'

connection_url=f'sqlite:///{db_url}'
engine=sql.create_engine(connection_url)

In [16]:
# option 1 using pandas
df=pd.read_csv('data.csv')
df.head()

,title,open_date,movie_link,poster_link,critic_score,critic_sentiment,audience_score,audience_sentiment
0,The Confession Musical,"Opened Feb 20, 2023",/m/the_confession_musical,https://resizing.flixster.com/Tb2DWzSI8qJkxJsd...,NaN,NaN,100.0,positive
1,Cocaine Bear,"Opened Feb 24, 2023",/m/cocaine_bear,https://resizing.flixster.com/ZWfK4g9Hs3ap7o6a...,NaN,NaN,75.0,positive
2,Jesus Revolution,"Opened Feb 24, 2023",/m/jesus_revolution,https://resizing.flixster.com/dtd4eV2Pec2cdFlx...,NaN,NaN,99.0,positive
3,Luther: The Fallen Sun,"Opened Feb 24, 2023",/m/luther_the_fallen_sun,https://resizing.flixster.com/acLTpoS_R1pDX7XN...,NaN,NaN,77.0,positive
4,Linoleum,"Opened Feb 24, 2023",/m/linoleum_2022,https://resizing.flixster.com/JLCsVZl_vbKJf6ua...,NaN,NaN,91.0,positive


In [ ]:
df.to_sql('movies', index=False, con=engine, if_exists='append')

In [50]:
# delete table
engine.execute('drop table movies')

In [52]:
# option 2 use SQL queries
# step 1
sql_query='''
create table movies(
    title varchar, 
    open_date varchar, 
    movie_link varchar, 
    poster_link varchar, 
    critic_score dec, 
    critic_sentiment varchar, 
    audience_score dec, 
    audience_sentiment varchar
);
'''

engine.execute(sql_query)

In [54]:
# step 2 insert
sql_query='''
insert into movies
values
'''

for idx, row in df.iterrows(): 
    sql_query=sql_query+str(tuple(row.fillna('').values))+', '

sql_query=sql_query[:-2]
# print(sql_query)
engine.execute(sql_query)

In [56]:
# delete table
engine.execute('drop table movies')

In [24]:
# option 3 ORM
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Float, Sequence, Integer
from sqlalchemy.orm import Session

In [25]:
Base=declarative_base() # use automapa_base if schema already exists

In [26]:
class Movie(Base): 
    __tablename__='movies'
    id=Column(Integer, primary_key=True)
    title=Column(String) 
    open_date=Column(String)
    movie_link=Column(String)
    poster_link=Column(String)
    critic_score=Column(String)
    critic_sentiment=Column(String)
    audience_score=Column(String)
    audience_sentiment=Column(String)

In [27]:
# check db at the connect (engine) to see if schema exists and/or matches
Base.metadata.create_all(engine)

In [28]:
import csv

id=1
session=Session(engine)

with open('data.csv') as f:
    csv_reader=csv.reader(f)
    header=next(csv_reader)
    # iterate through all rows
    for row in csv_reader: 
        print(f'Inserting {id}...')
        new_movie=Movie(
                id=id, 
                title=row[0], 
                open_date=row[1], 
                movie_link=row[2], 
                poster_link=row[3], 
                critic_score=row[4], 
                critic_sentiment=row[5], 
                audience_score=row[6], 
                audience_sentiment=row[7]
            )
        session.add(new_movie)
        id+=1
    
    # commit all records
    session.commit()
    session.close()

Inserting 1...
Inserting 2...
Inserting 3...
Inserting 4...
Inserting 5...
Inserting 6...
Inserting 7...
Inserting 8...
Inserting 9...
Inserting 10...
Inserting 11...
Inserting 12...
Inserting 13...
Inserting 14...
Inserting 15...
Inserting 16...
Inserting 17...
Inserting 18...
Inserting 19...
Inserting 20...
Inserting 21...
Inserting 22...
Inserting 23...
Inserting 24...
Inserting 25...
Inserting 26...
Inserting 27...
Inserting 28...
Inserting 29...
Inserting 30...
Inserting 31...
Inserting 32...
Inserting 33...
Inserting 34...
Inserting 35...
Inserting 36...
Inserting 37...
Inserting 38...
Inserting 39...
Inserting 40...
Inserting 41...
Inserting 42...
Inserting 43...
Inserting 44...
Inserting 45...
Inserting 46...
Inserting 47...
Inserting 48...
Inserting 49...
Inserting 50...
Inserting 51...
Inserting 52...
Inserting 53...
Inserting 54...
Inserting 55...
Inserting 56...
Inserting 57...
Inserting 58...
Inserting 59...
Inserting 60...
Inserting 61...
Inserting 62...
Inserting 63...
I

In [29]:
engine.execute('select * from movies').fetchall()

[(1, 'The Confession Musical', 'Opened Feb 20, 2023', '/m/the_confession_musical', 'https://resizing.flixster.com/Tb2DWzSI8qJkxJsdTobVNvgh2k4=/180x258/v2/https://resizing.flixster.com/2IhJFlEIRWGWyb-qsP4sTN0Wu6g=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzLzI2NGU3ZDQxLTU1YTUtNDBlNi05ZDdjLTMxYjllYTUyYjMwMS5qcGc=', '', '', 100.0, 'positive'),
 (2, 'Cocaine Bear', 'Opened Feb 24, 2023', '/m/cocaine_bear', 'https://resizing.flixster.com/ZWfK4g9Hs3ap7o6ac5nxwS-gtPU=/180x258/v2/https://resizing.flixster.com/HyxB3a8sdK-y6fUI4akiNAT39KQ=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2I1YjM3YTg1LTg5YTgtNGUzNC1iMjFmLTJkMmIwNmQ1ZTk0YS5qcGc=', '', '', 75.0, 'positive'),
 (3, 'Jesus Revolution', 'Opened Feb 24, 2023', '/m/jesus_revolution', 'https://resizing.flixster.com/dtd4eV2Pec2cdFlx5sl8mRE_K2s=/180x258/v2/https://resizing.flixster.com/FLuS1mtbLsmLtJCuxgvnN8PZ1Bg=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzLzU5ODQ1Y2M1LWY1MWEtNDIzOS04NDRjLTcwMTlkY2Q5MGE3NS5qcGc=', '', '', 99.0, 'positive'),
 (4, 'Luther: The Fallen Sun', 'Opened